In [9]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.layers.convolutional import Conv2D 
from keras.layers.convolutional import MaxPooling2D 
from keras.layers import Flatten 
from keras.datasets import mnist
import cv2
from PIL import Image

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

X_train = X_train / 255 
X_test = X_test / 255 

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1] 

def convolutional_model():
    model = Sequential()
    model.add(Conv2D(16, (5, 5), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(8, (2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) 
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    return model

model = convolutional_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=12, batch_size=200, verbose=2)

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))
model.save('my_mnist_model.h5')
#Custom Image Prediction
size = 28,28
image1="test.jpg"
im = Image.open(image1)
im_resized = im.resize(size, Image.ANTIALIAS)
im_resized.save("down.png","PNG")
    
img = cv2.imread("down.png",0)  
img = img / 255 
img = np.reshape(img,(1, 28, 28, 1)) 
model.predict_classes(img)[0]

Epoch 1/12
300/300 - 12s - loss: 0.5147 - accuracy: 0.8569 - val_loss: 0.1502 - val_accuracy: 0.9530
Epoch 2/12
300/300 - 12s - loss: 0.1320 - accuracy: 0.9609 - val_loss: 0.0903 - val_accuracy: 0.9732
Epoch 3/12
300/300 - 12s - loss: 0.0917 - accuracy: 0.9726 - val_loss: 0.0644 - val_accuracy: 0.9804
Epoch 4/12
300/300 - 12s - loss: 0.0729 - accuracy: 0.9783 - val_loss: 0.0607 - val_accuracy: 0.9804
Epoch 5/12
300/300 - 12s - loss: 0.0612 - accuracy: 0.9816 - val_loss: 0.0548 - val_accuracy: 0.9817
Epoch 6/12
300/300 - 12s - loss: 0.0542 - accuracy: 0.9836 - val_loss: 0.0451 - val_accuracy: 0.9860
Epoch 7/12
300/300 - 12s - loss: 0.0473 - accuracy: 0.9857 - val_loss: 0.0482 - val_accuracy: 0.9853
Epoch 8/12
300/300 - 12s - loss: 0.0414 - accuracy: 0.9876 - val_loss: 0.0447 - val_accuracy: 0.9848
Epoch 9/12
300/300 - 12s - loss: 0.0395 - accuracy: 0.9874 - val_loss: 0.0398 - val_accuracy: 0.9871
Epoch 10/12
300/300 - 12s - loss: 0.0351 - accuracy: 0.9895 - val_loss: 0.0496 - val_accura

6

In [60]:
from PIL import Image
import cv2
import numpy as np
from keras.models import load_model

# new_model=load_model('my_mnist_model.h5')
# size = 28,28
# file="7.jpg"
# image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
# image = cv2.resize(image, (28, 28))
# image = image.astype('float32')
# image = image.reshape(1, 28, 28, 1)
# image = 255-image
# image /= 255
# new_model.predict_classes(img)[0]



from PIL import Image
import cv2
import numpy as np
from keras.models import load_model

model = load_model('my_mnist_model.h5')

def predict(path_image):
    data = []
    im = cv2.imread(path_image)
    im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    ret,thre = cv2.threshold(im_gray,220,255,cv2.THRESH_BINARY)
    im_not = cv2.bitwise_not(thre)
    _,contours,hierachy = cv2.findContours(im_not,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        (x,y,w,h) = cv2.boundingRect(cnt)
        roi = im_not[y:y+h,x:x+w]
        if (roi.shape[0]<28) and (roi.shape[1]<28):
            pass
        else:
            roi_resize = cv2.resize(roi,(22,22),interpolation = cv2.INTER_LINEAR)
            roi_padding = np.pad(roi_resize,(3,3),'constant',constant_values=(0,0))
            kernel = np.array([[0,1]],np.uint8)
            roi_dila = cv2.dilate(roi_padding,kernel)
            roi_reshape = roi_dila.reshape((1,28,28,1)).astype(np.float32)/255
            predict = model.predict_classes(roi_reshape)
            cv2.putText(im,str(int(predict)),(int((x)),int((y))),1,cv2.FONT_HERSHEY_COMPLEX,(0,255,0),1,cv2.LINE_AA)
            data.append([x,y,str(int(predict))])
    answer_str=[]
    for i in data:
        answer_str.append(str(i[-1]))
    return (''.join(answer_str))
a=predict('2.jpg')
print(a)

2
